In [1]:
import spotipy
import re
from decouple import config
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import warnings
import seaborn as sns
import requests
import time
import bs4
import webdriver_manager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from IPython.display import clear_output
import time
from tqdm import tqdm
import pandas as pd
import IPython
import os
from decouple import config
import re
from IPython.display import clear_output
warnings.filterwarnings('ignore')

In [83]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [3]:
albums = pd.read_csv("data/albums.csv")
albums.head()

,album_id,album_name,total_tracks,release_date,album_uri,spotify_url,image_url,artist_id
0,4DvNMpnlo9GoWiTJWMfRAA,Ogo Chirabedanar Sathi - Hemanta Mukherjee,7,2022-05-04,spotify:album:4DvNMpnlo9GoWiTJWMfRAA,https://open.spotify.com/album/4DvNMpnlo9GoWiT...,https://i.scdn.co/image/ab67616d0000b27308b9a7...,02Um2HIOrUdsy3wqPBZwsj
1,2YOOgRzDIgOp0kdZACTVFc,Olo Soi Dekhey Jarey Dekhey Ja,17,2020-09-30,spotify:album:2YOOgRzDIgOp0kdZACTVFc,https://open.spotify.com/album/2YOOgRzDIgOp0kd...,https://i.scdn.co/image/ab67616d0000b2732c2ff2...,02Um2HIOrUdsy3wqPBZwsj
2,3aXxkiwm7EBXRXiM3xMcpX,Eso Amar Ghare,18,2020-01-06,spotify:album:3aXxkiwm7EBXRXiM3xMcpX,https://open.spotify.com/album/3aXxkiwm7EBXRXi...,https://i.scdn.co/image/ab67616d0000b273782940...,02Um2HIOrUdsy3wqPBZwsj
3,0ixRvfbmnmd4hBYCDzMljp,Moli (Original Motion Picture Soundtrack),8,2011-04-01,spotify:album:0ixRvfbmnmd4hBYCDzMljp,https://open.spotify.com/album/0ixRvfbmnmd4hBY...,https://i.scdn.co/image/ab67616d0000b273c8621b...,02Um2HIOrUdsy3wqPBZwsj
4,5b6jzg1UcSFNheady0Md24,Saikat Mukherjee With His Golden Harmonica,37,2008-11-01,spotify:album:5b6jzg1UcSFNheady0Md24,https://open.spotify.com/album/5b6jzg1UcSFNhea...,https://i.scdn.co/image/ab67616d0000b27340580f...,02Um2HIOrUdsy3wqPBZwsj


In [4]:
albums["album_name"]

0       Ogo Chirabedanar Sathi - Hemanta Mukherjee
1                   Olo Soi Dekhey Jarey Dekhey Ja
2                                   Eso Amar Ghare
3        Moli (Original Motion Picture Soundtrack)
4       Saikat Mukherjee With His Golden Harmonica
                           ...                    
1930                                       Routine
1931                    Faded (Lost Stories Remix)
1932                              Sing Me to Sleep
1933                               Faded (Remixes)
1934                                         Faded
Name: album_name, Length: 1935, dtype: object

In [84]:
def extract_name(name):
    name = re.sub(r' \(.*\)', '', name)
    name = re.sub(r'-.*', '', name)
    name = re.sub(r'feat.*', '', name)
    name = re.sub(r':.*', '', name)
    name = re.sub("Volume", "", name)
    name = re.search(r'([A-Za-z\s]+)', name)
    if name:
        name = name.group()
        return name.strip()
    # return name.strip()

In [85]:
album_names = albums["album_name"].apply(extract_name)
album_names

0                           Ogo Chirabedanar Sathi
1                   Olo Soi Dekhey Jarey Dekhey Ja
2                                   Eso Amar Ghare
3                                             Moli
4       Saikat Mukherjee With His Golden Harmonica
                           ...                    
1930                                       Routine
1931                                         Faded
1932                              Sing Me to Sleep
1933                                         Faded
1934                                         Faded
Name: album_name, Length: 1935, dtype: object

In [86]:
detected = []
for i in tqdm(range(20), desc="Detecting"):
    text = album_names[100+i]
    text = text.replace(" ", "%20")
    url = f"https://translate.google.co.in/?sl=auto&tl=en&text={text}&op=translate"
    driver.get(url)
    text_detected = ""
    j=0
    while True:
        text_detected = driver.find_element(By.CSS_SELECTOR, ".VfPpkd-AznF2e-LUERP-bN97Pc").text
        if "detected" in text_detected.lower() or j>10:
            break
        time.sleep(1)
        j+=1
    text_detected = text_detected.lower()
    if "detected" in text_detected:
        lang_detected = text_detected.split("-")[0].strip()
        detected.append(lang_detected)
    else:
        print("Not detected")

Detecting: 100%|██████████| 20/20 [01:06<00:00,  3.33s/it]


In [87]:
for i in range(len(detected)):
    print(f"{album_names[100+i]} - {detected[i]}")

Hemanta Mukherjee Protham Prodip - bengali
New Bengali Songs Edited From Old Source - english
Emon Diney Taare Bola Jaay - bengali
Gaaner Surer Dhara - bengali
Tuni Bou - arabic
Pratibha - english
Parashmoni - albanian
Bengali Modern Songs Munmun Ghosh - bengali
Ashirbad - bengali
Hemanta Mukherjee Rabindra Sangeet - gujarati
Rajeswari - malay
Surer Akashay - hindi
Surja Trishna - hindi
Pratiksha - hindi
Uttar Meleni - kannada
Kapalkundala - slovenian
Khana Baraha - hindi
Meghmukti - javanese
Subarna Golak - bengali
Sudurer Piyasi - bengali


In [55]:
text_detected = driver.find_element(By.CSS_SELECTOR, ".VfPpkd-AznF2e-LUERP-bN97Pc").text
text_detected = text_detected.lower()
print(text_detected)

hindi - detected
english
spanish
french


In [58]:
if "detected" in text_detected:
    lang_detected = text_detected.split("-")[0].strip()
    print(lang_detected.title())

Hindi


## Artist

In [5]:
artists = pd.read_csv("data/artists.csv")
artists.head()

,artist_id,artist_name,genres,popularity,artist_uri,followers,spotify_url,image_url
0,02Um2HIOrUdsy3wqPBZwsj,Hemant Kumar,"['classic bollywood', 'classic pakistani pop',...",52,spotify:artist:02Um2HIOrUdsy3wqPBZwsj,923384,https://open.spotify.com/artist/02Um2HIOrUdsy3...,https://i.scdn.co/image/ab67616d0000b2738c9945...
1,06HL4z0CvFAxyc27GXpf02,Taylor Swift,['pop'],94,spotify:artist:06HL4z0CvFAxyc27GXpf02,59135608,https://open.spotify.com/artist/06HL4z0CvFAxyc...,https://i.scdn.co/image/ab6761610000e5ebfcf7c3...
2,0C8ZW7ezQVs4URX5aX7Kqx,Selena Gomez,"['dance pop', 'pop', 'post-teen pop']",83,spotify:artist:0C8ZW7ezQVs4URX5aX7Kqx,37486770,https://open.spotify.com/artist/0C8ZW7ezQVs4UR...,https://i.scdn.co/image/ab6761610000e5eba5205a...
3,0GF4shudTAFv8ak9eWdd4Y,Kishore Kumar,"['classic bollywood', 'desi pop', 'filmi', 'mo...",70,spotify:artist:0GF4shudTAFv8ak9eWdd4Y,5374483,https://open.spotify.com/artist/0GF4shudTAFv8a...,https://i.scdn.co/image/ab6761610000e5ebc9ac92...
4,0QsbYX8XslSSUcztc6u4pO,Geeta Dutt,"['classic bollywood', 'classic pakistani pop',...",47,spotify:artist:0QsbYX8XslSSUcztc6u4pO,663551,https://open.spotify.com/artist/0QsbYX8XslSSUc...,https://i.scdn.co/image/ab67616d0000b273f6d093...


In [28]:
genres = artists.genres
artists = artists.artist_name

In [25]:
import ast

genres = genres.apply(lambda x: ast.literal_eval(str(x)))
unique_genres = set()
for i in genres:
   for j in i:
       unique_genres.add(j)

In [26]:
unique_genres

{'art pop',
 'australian country',
 'australian pop',
 'british soul',
 'broadway',
 'canadian pop',
 'canadian soundtrack',
 'chutney',
 'classic bollywood',
 'classic pakistani pop',
 'classic rock',
 'classic tollywood',
 'contemporary country',
 'country',
 'country dawn',
 'country road',
 'dance pop',
 'desi pop',
 'disney',
 'eau claire indie',
 'electro house',
 'electropop',
 'filmi',
 'ghazal',
 'glam rock',
 'hollywood',
 'indie folk',
 'k-pop',
 'lata',
 'melancholia',
 'mellow gold',
 'modern bollywood',
 'movie tunes',
 'neo mellow',
 'orchestral soundtrack',
 'pop',
 'pop rock',
 'pop soul',
 'post-teen pop',
 'r&b',
 'rabindra sangeet',
 'rock',
 'rock drums',
 'scandipop',
 'show tunes',
 'soft rock',
 'soundtrack',
 'sufi',
 'texas country',
 'uk pop',
 'vapor pop',
 'viral pop'}

In [35]:
genre_artist = {}
for i in unique_genres:
    genre_artist[i] = list(artists[genres.apply(lambda x: i in x)])

In [40]:
genre_artist

{'broadway': ['Alan Menken'],
 'lata': ['Lata Mangeshkar'],
 'r&b': ['Beyoncé'],
 'australian pop': ['Keith Urban'],
 'country': ['Keith Urban', 'Miranda Lambert', 'Maren Morris'],
 'canadian soundtrack': ['Christophe Beck'],
 'electropop': ['Billie Eilish'],
 'movie tunes': ['Alan Menken', "Paige O'Hara"],
 'soft rock': ['Phil Collins'],
 'texas country': ['Maren Morris'],
 'classic tollywood': ['Mohammed Rafi'],
 'country road': ['Keith Urban'],
 'british soul': ['Adele'],
 'rock drums': ['Phil Collins'],
 'glam rock': ['Queen', 'Freddie Mercury'],
 'canadian pop': ['Justin Bieber', 'Céline Dion', 'Shawn Mendes'],
 'rock': ['Queen', 'Freddie Mercury', 'Phil Collins', 'Christina Perri'],
 'pop': ['Hemant Kumar',
  'Taylor Swift',
  'Selena Gomez',
  'Kishore Kumar',
  'Geeta Dutt',
  'Mohammed Rafi',
  'Keith Urban',
  'Lady Gaga',
  'Justin Bieber',
  'Britney Spears',
  'Naomi Scott',
  'ROSÉ',
  'Lewis Capaldi',
  'Céline Dion',
  'Adele',
  'Mukesh',
  'Camila Cabello',
  'Asha Bh

In [37]:
hindi_genres = ['desi pop', "classic bollywood", 'modern bollywood', "ghazal", 'desi pop','filmi']

In [38]:
hindi_artists = []
for i in hindi_genres:
    hindi_artists.extend(genre_artist[i])
hindi_artists = list(set(hindi_artists))
len(hindi_artists)

7

In [39]:
hindi_artists

['Mukesh',
 'Hemant Kumar',
 'Lata Mangeshkar',
 'Asha Bhosle',
 'Mohammed Rafi',
 'Geeta Dutt',
 'Kishore Kumar']

In [41]:
language = []
for artist in artists:
    if artist in hindi_artists:
        language.append("hindi")
    else:
        language.append("english")

In [42]:
artists = pd.read_csv("data/artists.csv")
artists.head()

,artist_id,artist_name,genres,popularity,artist_uri,followers,spotify_url,image_url
0,02Um2HIOrUdsy3wqPBZwsj,Hemant Kumar,"['classic bollywood', 'classic pakistani pop',...",52,spotify:artist:02Um2HIOrUdsy3wqPBZwsj,923384,https://open.spotify.com/artist/02Um2HIOrUdsy3...,https://i.scdn.co/image/ab67616d0000b2738c9945...
1,06HL4z0CvFAxyc27GXpf02,Taylor Swift,['pop'],94,spotify:artist:06HL4z0CvFAxyc27GXpf02,59135608,https://open.spotify.com/artist/06HL4z0CvFAxyc...,https://i.scdn.co/image/ab6761610000e5ebfcf7c3...
2,0C8ZW7ezQVs4URX5aX7Kqx,Selena Gomez,"['dance pop', 'pop', 'post-teen pop']",83,spotify:artist:0C8ZW7ezQVs4URX5aX7Kqx,37486770,https://open.spotify.com/artist/0C8ZW7ezQVs4UR...,https://i.scdn.co/image/ab6761610000e5eba5205a...
3,0GF4shudTAFv8ak9eWdd4Y,Kishore Kumar,"['classic bollywood', 'desi pop', 'filmi', 'mo...",70,spotify:artist:0GF4shudTAFv8ak9eWdd4Y,5374483,https://open.spotify.com/artist/0GF4shudTAFv8a...,https://i.scdn.co/image/ab6761610000e5ebc9ac92...
4,0QsbYX8XslSSUcztc6u4pO,Geeta Dutt,"['classic bollywood', 'classic pakistani pop',...",47,spotify:artist:0QsbYX8XslSSUcztc6u4pO,663551,https://open.spotify.com/artist/0QsbYX8XslSSUc...,https://i.scdn.co/image/ab67616d0000b273f6d093...


In [43]:
artists["language"] = language

In [44]:
artists.to_csv("data/artists.csv", index=False)

In [45]:
artists["language"].value_counts()

english    33
hindi       7
Name: language, dtype: int64

In [7]:
token = spotipy.util.prompt_for_user_token(client_id= config("SPOTIFYID"), client_secret= config("SPOTIFYPASSWORD"), redirect_uri='http://mysite.com/callback/', 
    scope='playlist-read-private user-modify-playback-state playlist-modify-public user-read-recently-played user-follow-read user-follow-modify')
sp = spotipy.Spotify(auth=token)

In [8]:
USER = "392uowv8fh00b9ie6ccn9cegs"

In [9]:
following = sp.current_user_followed_artists(limit=50, after=None)

In [10]:
artists_raw = pd.DataFrame(following['artists']['items'])
artists_raw.head(2).T

,0,1
external_urls,{'spotify': 'https://open.spotify.com/artist/0...,{'spotify': 'https://open.spotify.com/artist/0...
followers,"{'href': None, 'total': 1059591}","{'href': None, 'total': 64546282}"
genres,"[classic bollywood, classic pakistani pop, fil...",[pop]
href,https://api.spotify.com/v1/artists/02Um2HIOrUd...,https://api.spotify.com/v1/artists/06HL4z0CvFA...
id,02Um2HIOrUdsy3wqPBZwsj,06HL4z0CvFAxyc27GXpf02
images,"[{'height': 640, 'url': 'https://i.scdn.co/ima...","[{'height': 640, 'url': 'https://i.scdn.co/ima..."
name,Hemant Kumar,Taylor Swift
popularity,54,100
type,artist,artist
uri,spotify:artist:02Um2HIOrUdsy3wqPBZwsj,spotify:artist:06HL4z0CvFAxyc27GXpf02


In [46]:
len(artists_raw)

40

In [17]:
artist_id = "4PULA4EFzYTrxYvOVlwpiQ"
artist = sp.artist(artist_id)
artist

{'external_urls': {'spotify': 'https://open.spotify.com/artist/4PULA4EFzYTrxYvOVlwpiQ'},
 'followers': {'href': None, 'total': 13377664},
 'genres': ['desi hip hop', 'pakistani hip hop', 'punjabi hip hop'],
 'href': 'https://api.spotify.com/v1/artists/4PULA4EFzYTrxYvOVlwpiQ',
 'id': '4PULA4EFzYTrxYvOVlwpiQ',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5eb65d54c88ab4c1abf204b360f',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab6761610000517465d54c88ab4c1abf204b360f',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f17865d54c88ab4c1abf204b360f',
   'width': 160}],
 'name': 'Sidhu Moose Wala',
 'popularity': 80,
 'type': 'artist',
 'uri': 'spotify:artist:4PULA4EFzYTrxYvOVlwpiQ'}

In [14]:
artist_albums = sp.artist_albums(artist_id, album_type='album')
album1 = artist_albums["items"][0]

In [16]:
len(album1["available_markets"])

183